In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import numpy as np
import torch
from skimage.color import rgb2lab, rgb2gray
from torchvision import datasets, transforms

class GrayscaleImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        global img_original
        global targets
        
        path, target = self.imgs[index]
        img = self.loader(path)
        img = self.transform(img)
        
        if self.transform is not None:
#             print(path.find('train'))
            if path.find('test') != -1:
                img_original = np.asarray(img).transpose((2, 1, 0)) / 255
                targets = int(path[43:-4])
            else:
                img = self.transform(img)
                original_targets = path[44:-4]
#                 print(original_targets)
                targets = np.zeros((5))
                for i in range(5):
                    targets[i] = int(ord(original_targets[i]) - 1)
                img_original = np.asarray(img).transpose((2, 1, 0)) / 255
                targets = np.asarray(targets).astype(int)
        
        return img_original, targets

class open_train():
    def __init__(self, path, batch_size):
        self.path = path
        self.batch_size = batch_size
    def get_img_loader(self):
        train_transforms = transforms.Compose([
            transforms.Resize((112, 560)),
        ])
        train_imagefolder = GrayscaleImageFolder(
            self.path,
            train_transforms
        )
        train_loader = torch.utils.data.DataLoader(
            train_imagefolder,
            batch_size=self.batch_size,
            shuffle=True
        )
        return  train_loader
class open_val():
    def __init__(self, path, batch_size):
        self.path = path
        self.batch_size = batch_size
    def get_img_loader(self):
        train_transforms = transforms.Compose([
            transforms.Resize((112, 560)),
        ])
        val_imgefolder = GrayscaleImageFolder(
            self.path,
            train_transforms
        )
#         val_imgefolder.imgs = sorted(val_imgefolder.imgs, key = lambda x:)
        val_loader = torch.utils.data.DataLoader(
            val_imgefolder,
            batch_size=self.batch_size,
            shuffle=False
        )
        return  val_loader

#%%

import torch.nn as nn
import torchvision.models as model
# model.resnet18()
class Generator(nn.Module):
    def __init__(self, num_class = 122):
        super(Generator, self).__init__()
        self.layer_1 = nn.Sequential(
            model.resnet18(num_classes = num_class)
        )
        self.layer_2 = nn.Sequential(
            nn.Sigmoid()
        )
    def forward(self, input):
        x_ = self.layer_1(input)
#         x_ = self.layer_2(x_)
        return x_


import torch
import torch.nn as nn
import numpy as np
import torchvision.transforms as T

import csv

import pandas as pd

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

G = Generator().to(DEVICE)

criterion_BCE = nn.BCELoss()
criterion_MSE = nn.MSELoss()
criterion_L1Loss = nn.L1Loss()
criterion_CEL = nn.CrossEntropyLoss()

G_opt = torch.optim.Adam(G.parameters(), lr=0.02, betas=(0.5, 0.999))

batch_size = 256

max_epoch = 15

train = open_train('/kaggle/input/third-data/images/train/', batch_size)
train_loader = train.get_img_loader()
test = open_val('/kaggle/input/third-data/images/test/',batch_size)
test_loader = test.get_img_loader()

def f_train(train_loader, G, criterion_CEL, criterion_MSE, criterion_L1Loss, G_opt, DEVICE):
    step = 0
    for epoch in range(max_epoch):
        for idx, (train_data, target) in enumerate(train_loader):
            for i in range(5):
                if DEVICE:
                    train_set = train_data[:,:,i * 112:(i + 1) * 112,:].type(torch.FloatTensor).to(DEVICE)
                    target_set = target[...,i].to(DEVICE)
                
                output = G(train_set)
                G_loss = criterion_CEL(output, target_set)
                
                G.zero_grad()
                G_loss.backward()
                G_opt.step()

                if step % 10 == 0:
                    print('Epoch: {}/{}, Step: {}, G Loss: {}'.format(
                        epoch, max_epoch,
                        step * 256,
                        G_loss.item()
                        )
                    )
                step += 1

def create_csv(test_arr):
    test_lables = pd.DataFrame(test_arr)
    test_lables.to_csv('/kaggle/working/a.csv')
    
# from PIL import Image

# dirlists = os.listdir('imges/trian/class')
# dirlists = sorted(dirlists, key=lambda x : int(x[:-4]))

# def f_val(dirlists, G, DEVICE):
#     for dirlist in dirlists:
#         image_data = Image.open('imges/trian/class' + dirlist)
#         iamge_tensor = T.ToTensor()(image_data) / 255
    
def f_val(test_loader, G, DEVICE):
    data_dict = {}
    global test_set
    for idx, (test_data, target) in enumerate(test_loader):
        arr_all = []
        for i in range(5):
            arr = []
            if DEVICE:
                test_set = test_data[:, :, i * 112:(i + 1) * 112, :].type(torch.FloatTensor).to(DEVICE)

            outputs = G(test_set)
            outputs = outputs.cpu().data.numpy()
            for j, output in enumerate(outputs):
                temp = np.argmax(output)
                arr.append(chr(int(temp) + 1))
            arr_all.append(arr)
        
        for j in range(len(arr_all[0])):
            temp_1 = ''
            for k in range(5):
                temp_1 += arr_all[k][j]
            data_dict[int(target[j])] = temp_1
    sort_data = sorted(data_dict.items(), key=lambda x: x[0])
    create_csv(dict(sort_data).values())
    
f_train(train_loader, G, criterion_CEL, criterion_MSE, criterion_L1Loss, G_opt, DEVICE)
f_val(test_loader, G, DEVICE)